## Dutch article cleaning

In [ ]:
import re
import os

def clean_article_start(text):
    # removing tokens in the start of the  article
    text = re.sub(r"^(Art\.?|Artikel|ANNEXE|DROIT FUTUR|Antérieurement|Voir note sous TITRE|BIJLAGE|Inbreuk op artikel|Voorheen)\s*", "", text, flags=re.IGNORECASE)

    # removing numbers, dots, non-words
    text = re.sub(r"^\d+[^\w]*\s*", "", text)

    # list of common words in the beginning
    forbidden_words = ["Art", "Artikel", "ANNEXE", "DROIT FUTUR", "Antérieurement", "Voir note sous TITRE", "BIJLAGE", "Inbreuk op artikel", "Voorheen"]

    # scanning for capital letter
    for match in re.finditer(r"[A-Z]", text):
        start_index = match.start()

        # excluding capital letter if inside a common word
        window_start = max(0, start_index - 10)
        window_text = text[window_start:start_index + 10]

        if any(forbidden.lower() in window_text.lower() for forbidden in forbidden_words):
            continue

        # checking next two characters after capital letter
        next_chars = text[start_index+1:start_index+3]
        if not re.match(r"[\s*'a-zA-Z]{1,2}", next_chars):
            continue

        return text[start_index:].strip()

    # If nothing found → return original text
    return text.strip()


# ---> REMOVING LONG ARTICLES BEFORE CLEANING <--- Can take the next 3 lines of code out when working with the whole corpus
with open("long_article_ids.json", "r", encoding="utf-8") as f:
    long_article_ids = json.load(f)

df_corpus_nl = df_corpus_nl[~df_corpus_nl["id"].isin(long_article_ids)]

# apply cleaning
df_corpus_nl["article_cleaned"] = df_corpus_nl["article"].apply(clean_article_start)

# making two directories for cleaned corpus and mixed corpus for comparison
os.makedirs("data/original_cleaned_mix_corpus", exist_ok=True)
os.makedirs("data/cleaned_corpus", exist_ok=True)

df_corpus_nl.to_csv("data/original_cleaned_mix_corpus/original_cleaned_mix_nl_corpus.csv", index=False)


df_corpus_nl_original_format = df_corpus_nl[["id", "reference", "article_cleaned"]].rename(columns={"article_cleaned": "article"})
df_corpus_nl_original_format.to_csv("data/cleaned_corpus/corpus_nl_cleaned.csv", index=False)

print("Created a mixed CSV of original and cleaned Dutch article texts for comparison.")

print("Saved cleaned Dutch corpus as CSV.")

## French article cleaning

In [ ]:
import re
import os

def clean_article_start(text):
    # removing tokens in the start of the  article
    text = re.sub(r"[\(\[]\s*(ancien article|ancien art|erronément intitulé art\.?)\s*\d+[^\]\)]*[\)\]]", " ", text, flags=re.IGNORECASE)
    text = re.sub(r"^(Art\.?|Article|ANNEXE|DROIT FUTUR|Antérieurement|Voir note sous TITRE|ancien article|Infraction à l'article)\s*", "", text, flags=re.IGNORECASE)

    # removing numbers, dots, non-words
    text = re.sub(r"^\d+[^\w]*\s*", "", text)

    # list of common words in the beginning
    forbidden_words = ["Art", "Article", "ANNEXE", "DROIT FUTUR", "Antérieurement", "Voir note sous TITRE", "ancien article", "Infraction à l'article"]

    # scanning for capital letter
    for match in re.finditer(r"[A-Z]", text):
        start_index = match.start()

        # excluding capital letter if inside a common word
        window_start = max(0, start_index - 10)
        window_text = text[window_start:start_index + 10]

        if any(forbidden.lower() in window_text.lower() for forbidden in forbidden_words):
            continue

        # checking next two characters after capital letter
        next_chars = text[start_index+1:start_index+3]
        if not re.match(r"[\s*'a-zA-Z]{1,2}", next_chars):
            continue

        return text[start_index:].strip()

    return text.strip()

# ---> REMOVING LONG ARTICLES BEFORE CLEANING <--- Can take the next 3 lines of code out when working with the whole corpus
with open("long_article_ids.json", "r", encoding="utf-8") as f:
    long_article_ids = json.load(f)

df_corpus_fr = df_corpus_fr[~df_corpus_fr["id"].isin(long_article_ids)]
# apply cleaning
df_corpus_fr["article_cleaned"] = df_corpus_fr["article"].apply(clean_article_start)

# making two directories for cleaned corpus and mixed corpus for comparison
os.makedirs("data/original_cleaned_mix_corpus", exist_ok=True)
os.makedirs("data/cleaned_corpus", exist_ok=True)

df_corpus_fr.to_csv("data/original_cleaned_mix_corpus/original_cleaned_mix_fr_corpus.csv", index=False)

df_corpus_fr_original_format = df_corpus_fr[["id", "reference", "article_cleaned"]].rename(columns={"article_cleaned": "article"})
df_corpus_fr_original_format.to_csv("data/cleaned_corpus/corpus_fr_cleaned.csv", index=False)

print("Created a mixed CSV of original and cleaned French article texts for comparison.")
print("Saved cleaned French corpus as CSV.")

## SAMPLING CODES (BM25)

In [ ]:
from datasets import load_from_disk

corpus = load_from_disk("/content/data/cleaned/corpus")
test = load_from_disk("/content/data/cleaned/test")

corpus_fr = corpus['fr']
test_fr = test['fr']


### WITH SCORES AND RANKS FOR FRENCH

from datasets import load_from_disk
from rank_bm25 import BM25Okapi
from tqdm import tqdm
import json
import os
from nltk.tokenize.toktok import ToktokTokenizer

# Initialize tokenizer
tokenizer = ToktokTokenizer()

# Load cleaned corpus and test queries (French as example)
corpus = load_from_disk("data/cleaned/corpus")['fr']
test_queries = load_from_disk("data/cleaned/test")['fr']

# Prepare corpus documents and their IDs
corpus_docs = [doc['article'] for doc in corpus]
corpus_ids = [str(doc['id']) for doc in corpus]

# Tokenize corpus with ToktokTokenizer
tokenized_corpus = [tokenizer.tokenize(doc.lower()) for doc in corpus_docs]
bm25 = BM25Okapi(tokenized_corpus)

# Prepare output
output = []

for query in tqdm(test_queries):

    query_id = query['id']
    query_text = query['question']
    
    # Get relevant article IDs for this query
    relevant_ids = [id_.strip() for id_ in query['article_ids'].split(",") if id_.strip() != ""]
    num_relevant = len(relevant_ids)
    
    # Calculate how many negatives needed
    num_negatives_needed = 100 - num_relevant
    
    # Tokenize query text using ToktokTokenizer
    tokenized_query = tokenizer.tokenize(query_text.lower())

    # BM25 scoring
    bm25_scores = bm25.get_scores(tokenized_query)
    
    # Combine document ID, score and index
    scored_docs = [
        {"doc_id": corpus_ids[idx], "score": float(bm25_scores[idx]), "rank": None}
        for idx in range(len(bm25_scores))
    ]
    
    # Sort by score (high to low) → rank them properly
    scored_docs = sorted(scored_docs, key=lambda x: x["score"], reverse=True)
    
    # Add final rank after sorting (rank 0 = highest score)
    for final_rank, doc in enumerate(scored_docs):
        doc["rank"] = final_rank + 1  # make ranks 1-based

    # Select negatives (skip relevant ids)
    hard_negatives = []
    
    for doc in scored_docs:
        if doc["doc_id"] not in relevant_ids:
            hard_negatives.append(doc)
        if len(hard_negatives) >= num_negatives_needed:
            break

    # Save full ranked list + hard negatives
    output.append({
        "query_id": query_id,
        "query_text": query_text,
        "relevant_ids": relevant_ids,
        "bm25_ranked_list": scored_docs,  # FULL ranked list with doc_id, score, rank
        "hard_negatives": hard_negatives  # selected hard negatives only
    })

# Save to JSONL
os.makedirs("data/bm25_sampling", exist_ok=True)
output_path = "data/bm25_sampling/bm25_with_scores_and_ranks_fr.jsonl"

with open(output_path, "w", encoding="utf-8") as f:
    for entry in output:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

#print("✅ BM25 sampling complete and saved (with scores and ranks).")



FileNotFoundError: [Errno 2] No such file or directory: 'bm25_hard_negatives_fr.jsonl'

In [ ]:
### WITH SCORES AND RANKS FOR DUTCH

from datasets import load_from_disk
from rank_bm25 import BM25Okapi
from tqdm import tqdm
import json
import os
from nltk.tokenize.toktok import ToktokTokenizer

# Initialize tokenizer
tokenizer = ToktokTokenizer()

# Load cleaned corpus and test queries (Dutch)
corpus = load_from_disk("data/cleaned/corpus")['nl']
test_queries = load_from_disk("data/cleaned/test")['nl']

# Prepare corpus documents and their IDs
corpus_docs = [doc['article'] for doc in corpus]
corpus_ids = [str(doc['id']) for doc in corpus]

# Tokenize corpus with ToktokTokenizer
tokenized_corpus = [tokenizer.tokenize(doc.lower()) for doc in corpus_docs]
bm25 = BM25Okapi(tokenized_corpus)

# Prepare output
output = []

for query in tqdm(test_queries):

    query_id = query['id']
    query_text = query['question']
    
    # Get relevant article IDs for this query
    relevant_ids = [id_.strip() for id_ in query['article_ids'].split(",") if id_.strip() != ""]
    num_relevant = len(relevant_ids)
    
    # Calculate how many negatives needed
    num_negatives_needed = 100 - num_relevant
    
    # Tokenize query text using ToktokTokenizer
    tokenized_query = tokenizer.tokenize(query_text.lower())

    # BM25 scoring
    bm25_scores = bm25.get_scores(tokenized_query)
    
    # Combine document ID, score and index
    scored_docs = [
        {"doc_id": corpus_ids[idx], "score": float(bm25_scores[idx]), "rank": None}
        for idx in range(len(bm25_scores))
    ]
    
    # Sort by score (high to low) → rank them properly
    scored_docs = sorted(scored_docs, key=lambda x: x["score"], reverse=True)
    
    # Add final rank after sorting (rank 0 = highest score)
    for final_rank, doc in enumerate(scored_docs):
        doc["rank"] = final_rank + 1  # make ranks 1-based

    # Select negatives (skip relevant ids)
    hard_negatives = []
    
    for doc in scored_docs:
        if doc["doc_id"] not in relevant_ids:
            hard_negatives.append(doc)
        if len(hard_negatives) >= num_negatives_needed:
            break

    # Save full ranked list + hard negatives
    output.append({
        "query_id": query_id,
        "query_text": query_text,
        "relevant_ids": relevant_ids,
        "bm25_ranked_list": scored_docs,  # FULL ranked list with doc_id, score, rank
        "hard_negatives": hard_negatives  # selected hard negatives only
    })

# Save to JSONL
os.makedirs("data/bm25_sampling", exist_ok=True)
output_path = "data/bm25_sampling/bm25_with_scores_and_ranks_nl.jsonl"

with open(output_path, "w", encoding="utf-8") as f:
    for entry in output:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

print("BM25 sampling complete and saved (with scores and ranks) for DUTCH.")

In [ ]:
### FOR LOADING AND CHECKING


import json

# Load French BM25 results
fr_path = "data/bm25_sampling/bm25_with_scores_and_ranks_fr.jsonl"
with open(fr_path, "r", encoding="utf-8") as f:
    french_data = [json.loads(line) for line in f]

print(f"✅ Loaded {len(french_data)} French BM25 entries")
print("Example:")
print(french_data[:5])  # Show first entry

# Load Dutch BM25 results
nl_path = "data/bm25_sampling/bm25_hard_negatives_nl.jsonl"
with open(nl_path, "r", encoding="utf-8") as f:
    dutch_data = [json.loads(line) for line in f]

print(f"\n✅ Loaded {len(dutch_data)} Dutch BM25 entries")
print("Example:")
print(dutch_data[5])  # Show first entry

## The following code is for removing all the relevant article ids from the data. It can be used but I will not use it here

In [ ]:
## Cleaning and saving as CSV and dataset.arrows
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import json
import os


ds_test = load_dataset("clips/bBSARD", "test")
test_fr = ds_test["fr"]
test_nl = ds_test["nl"]

# Load long article citation map
with open("queries_citing_long_articles.json", "r", encoding="utf-8") as f:
    queries_citing_long_articles = json.load(f)

long_articles_fr = queries_citing_long_articles["fr"]
long_articles_nl = queries_citing_long_articles["nl"]

# Cleaning function
def clean_query_set(test_set, long_article_map, lang_label):
    cleaned = []
    for query in test_set:
        query_id = str(query["id"])
        relevant_ids = [id_.strip() for id_ in query["article_ids"].split(",")]

        if len(relevant_ids) > 10:
            continue

        if query_id in long_article_map:
            long_ids = set(long_article_map[query_id])
            relevant_ids = [id_ for id_ in relevant_ids if id_ not in long_ids]

        if len(relevant_ids) == 0:
            continue

        query_cleaned = dict(query)
        query_cleaned["article_ids"] = ", ".join(relevant_ids)
        cleaned.append(query_cleaned)
    print(f"Cleaned {len(cleaned)} {lang_label} queries.")
    return cleaned

cleaned_fr = clean_query_set(test_fr, long_articles_fr, "French")
cleaned_nl = clean_query_set(test_nl, long_articles_nl, "Dutch")

os.makedirs("data/cleaned_queries_csv", exist_ok=True)
pd.DataFrame(cleaned_fr).to_csv("data/cleaned_queries_csv/cleaned_test_queries_fr.csv", index=False)
pd.DataFrame(cleaned_nl).to_csv("data/cleaned_queries_csv/cleaned_test_queries_nl.csv", index=False)

ds_cleaned_fr = Dataset.from_list(cleaned_fr)
ds_cleaned_nl = Dataset.from_list(cleaned_nl)

ds_cleaned = DatasetDict({
    "fr": ds_cleaned_fr,
    "nl": ds_cleaned_nl
})
os.makedirs("data/cleaned_queries_ds", exist_ok=True)
ds_cleaned.save_to_disk("data/cleaned_queries_ds/cleaned_test_queries")
print("Saved cleaned test queries to HuggingFace dataset format.")